# Loading the data

https://www.kaggle.com/datasets/jhoward/lsun_bedroom/data

@misc{yu2016lsun,
      title={LSUN: Construction of a Large-scale Image Dataset using Deep Learning with Humans in the Loop}, 
      author={Fisher Yu and Ari Seff and Yinda Zhang and Shuran Song and Thomas Funkhouser and Jianxiong Xiao},
      year={2016},
      eprint={1506.03365},
      archivePrefix={arXiv},
      primaryClass={cs.CV}
}

In [22]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
from torchvision import models
from torchvision.transforms import v2
from torchvision.datasets import ImageFolder

from tqdm import tqdm

In [57]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [16]:
data_path = './data/subset'
# data_path = './data/data0/lsun/bedroom'

In [17]:
# image size after transformations
image_size = 64

simple_load = v2.Compose([
    v2.Resize((image_size, image_size)),
    v2.PILToTensor(),
    v2.ToDtype(torch.float32),
    v2.Normalize([0.5], [0.5]),
])
dataset = ImageFolder(data_path, transform=simple_load)

# Training the models

### Gaussian Diffusion

In [4]:
from denoising_diffusion_pytorch import Unet, GaussianDiffusion, Trainer

/home/maciej/szkola/magisterka/sem1/Deep_Learning/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
model = Unet(
    dim = 64,
    dim_mults = (1, 2, 4, 8),
).cuda()

In [6]:
diffusion = GaussianDiffusion(
    model,
    image_size = 128,
    timesteps = 1000,   # number of steps
).cuda()

In [8]:
trainer = Trainer(
    diffusion,
    data_path,
    train_batch_size=16,
    train_lr=2e-5,
    train_num_steps=1,         # total training steps
    gradient_accumulate_every=2,    # gradient accumulation steps
    ema_decay=0.995,                # exponential moving average decay
    amp=True                        # turn on mixed precision
).cuda()

In [9]:
trainer.train()

  0%|          | 0/1 [00:00<?, ?it/s]

### DDPM

https://huggingface.co/docs/diffusers/en/tutorials/basic_training

In [18]:
from diffusers import DDPMPipeline, DDPMScheduler, UNet2DModel
from diffusers.utils import make_image_grid
from datasets import load_dataset

In [ ]:
dataset = load_dataset('data/subset/0', split='train')
dataset.set_transform(simple_load)

In [19]:
unet_model = UNet2DModel(
    sample_size=image_size,  # the target image resolution
    in_channels=3,  # the number of input channels, 3 for RGB images
    out_channels=3,  # the number of output channels
    layers_per_block=2,  # how many ResNet layers to use per UNet block
    block_out_channels=(128, 128, 256, 256, 512, 512),  # the number of output channels for each UNet block
    down_block_types=(
        "DownBlock2D",  # a regular ResNet downsampling block
        "DownBlock2D",
        "DownBlock2D",
        "DownBlock2D",
        "AttnDownBlock2D",  # a ResNet downsampling block with spatial self-attention
        "DownBlock2D",
    ),
    up_block_types=(
        "UpBlock2D",  # a regular ResNet upsampling block
        "AttnUpBlock2D",  # a ResNet upsampling block with spatial self-attention
        "UpBlock2D",
        "UpBlock2D",
        "UpBlock2D",
        "UpBlock2D",
    ),
)
unet_model.to(device)

In [ ]:
def evaluate(epoch, pipeline, random_state: int | None = None):
    # Sample some images from random noise (this is the backward diffusion process).
    # The default pipeline output type is `List[PIL.Image]`
    images = pipeline(
        batch_size=16,
        generator=torch.manual_seed(random_state),
    ).images

    # Make a grid out of the images
    image_grid = make_image_grid(images, rows=4, cols=4)

    # Save the images
    test_dir = os.path.join('saved', 'ddpm_training')
    os.makedirs(test_dir, exist_ok=True)
    image_grid.save(f"{test_dir}/{epoch:04d}.png")

In [37]:
def train_model(model, n_epochs: int, noise_scheduler, optimizer, dataset, batch_size: int = 16):
    data_loader = DataLoader(dataset, batch_size)

    for epoch in tqdm(range(n_epochs)):

        for batch in data_loader:

            # move to cuda/cpu
            batch.to(device)
            clean_images = batch["image"]

            # Sample noise to add to the images
            noise = torch.randn(clean_images.shape, device=clean_images.device)
            bs = clean_images.shape[0]

            # Sample a random timestep for each image
            timesteps = torch.randint(
                0, noise_scheduler.config.num_train_timesteps, (bs,), device=clean_images.device,
                dtype=torch.int64
            )

            # Add noise to the clean images according to the noise magnitude at each timestep
            # (this is the forward diffusion process)
            noisy_images = noise_scheduler.add_noise(clean_images, noise, timesteps)

            # Predict the noise residual
            noise_pred = model(noisy_images, timesteps, return_dict=False)[0]
            loss = F.mse_loss(noise_pred, noise)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        # evaluate every 5th epoch
        if (epoch + 1) % 5:
            # save model
            model_save_dir = os.path.join('saved', 'ddpm_training')
            os.makedirs(model_save_dir, exist_ok=True)
            torch.save(model.state_dict(), f'{model_save_dir}/{epoch:04d}.pth')
            # evaluate and save images
            pipeline = DDPMPipeline(unet=model, scheduler=noise_scheduler)
            evaluate(epoch, pipeline, random_state=epoch)

In [48]:
noise_scheduler = DDPMScheduler(num_train_timesteps=1000)

In [49]:
train_model(
    unet_model, 
    n_epochs=50, 
    noise_scheduler=noise_scheduler, 
    optimizer=optim.Adam(unet_model.parameters()), 
    dataset=dataset, 
    batch_size=16,
)

100%|██████████| 1/1 [00:13<00:00, 13.36s/it]
